# Семинар 1-6: Крупноблочная $HP$-модель

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import Bio.PDB as pdb

from hp_model.utils.sequence import HPSequenceManager
from hp_model.lattice import PseudoTriangularLattice, FCC3DLattice

# 0. Подготовка

На этом семинаре будем работать с 2D $HP$-моделью. 

И сразу прыгнем с места в карьер - прочитаем белок 1FSD и сделаем из него $HP$-последовательность:

In [ ]:
fpath = pdb.PDBList().retrieve_pdb_file("1fsd", 
                                        file_format="pdb")

In [ ]:
chain = pdb.PDBParser().get_structure("s", fpath)[0].child_list[0]

In [ ]:
hp_sequence = HPSequenceManager(pdb_chain=chain).get_hp_sequence()

In [ ]:
hp_sequence

Получим координаты всех $C_\alpha$:

In [ ]:
ca_coords = np.array([res["CA"].coord for res in chain])

# 1. Построение максимально плотного ядра

In [ ]:
from hp_model.prediction.hcore_generation.lp_generation import make_cores
from hp_model.plotting import Plotter

Сколько $H$-мономеров в $HP$-последовательности?

In [ ]:
# TODO?

Сгенерируем максимально плотные ядра для данной $HP$-последовательности:

In [ ]:
cores = make_cores(hp_sequence, None, False)

Сколько получилось ядер? Отрисуем их и визуально сравним!

In [ ]:
fig, axs = plt.subplots(1, len(cores), figsize=(15, 15 / len(cores)))

for i, core in enumerate(cores):
    Plotter.plot_hcore(axs[i], 
                       core)
    axs[i].axis("equal")
    axs[i].axis("off")

plt.show()

**Вопрос для обсуждения**: Если ядер получилось несколько, разные ли они?

К слову: <a href="https://projectswhynot.site/draggable_lattice_points/">давайте поиграем 🙂</a>

Посмотрим внимательнее на $HP$-последовательность. Может ли быть такое, что **ни в одно из этих ядер** белок 1FSD не фитится? 🤔

Разделим блоки, состоящие из нескольких последовательных $P$ и $H$ в $HP$-последовательности друг от друга:

In [ ]:
print(hp_sequence.replace("HP", "H-P").replace("PH", "P-H"))

**Вопрос для обсуждения**: Можно ли здесь что-то увидеть? 🤔

**Напоминание**: в $H$-ядре не могут находиться $P$-мономеры!

# 2. Оценка максимального числа контактов в $H$-ядре для данной $HP$-последовательности

Сколько контактов в полученных ядрах?

In [ ]:
for core in cores:
    # your code here

Мы поняли, что в таких ядрах разместить белок нельзя. А сколько контактов может быть в ядре, в котором разместить белок **можно**?

## 2.1. Посчитаем "периметр" $H$-ядра:

Можно заметить, что ядро - планарный граф с $N$ вершинами, $K$ ребрами (контактами) и $G$ гранями, $(G - 1)$ из которых треугольные. Обозначим число вершим на поверхности $H$-ядра как $R$.

Тогда $N - K + G = 2$; (по ф-ле Эйлера)

$2K = 3(G - 1) + R$;

Тогда $R = 3N - K - 3$,

а число точек **внутри** $H$-ядра: $N_{in} = N - R = K - 2N + 3$

Вычислим $R$ и $N_{in}$ для первого максимально плотного ядра:

In [ ]:
N = hp_sequence.count("H")

In [ ]:
# your code here

## 2.2. А есть ли в $HP$-последовательности такой $H$-мономер, который мог бы находиться внутри $H$-ядра (быть внутренней точкой)? 🤔

In [ ]:
from hp_model.prediction.protein_fitting.csp_fitting.csp import ConstraintBuilder

In [ ]:
check_verdict = ConstraintBuilder(hp_sequence, cores[0], PseudoTriangularLattice()).check_feasibility(verbose=True)

Если значение check_verdict["max_H-P_dist"] $\leq$ $N_{in}$, то нет такого мономера!

In [ ]:
# your code here

$\rightarrow$ 😕

## 2.3. Тогда какое число контактов допустимо? 🤔

In [ ]:
from hp_model.hcore.filtering._2d.kmax_filtering import get_Kmax

In [ ]:
K_max = get_Kmax(hp_sequence)
print(K_max)

Сделаем ядра, подходящие под это требование!

In [ ]:
# your code here

И снова их отрисуем:

In [ ]:
fig, axs = plt.subplots(1, len(cores), figsize=(15, 15 / len(cores)))

for i, core in enumerate(cores):
    Plotter.plot_hcore(axs[i], 
                       core)
    axs[i].axis("equal")
    axs[i].axis("off")

plt.show()

$\rightarrow$ Лучше? Лууучше 😉

## 3. Разместим белок в ядре!

Вообще, правильно проверить возможность размещения белка в каждом из сгенерированных ядер.

...но мы ограничимся первым 🙂 Его достаточно!

In [ ]:
from hp_model.prediction.protein_fitting.csp_fitting.fitting import fit_protein

In [ ]:
# your code here

Отрисуем, что получилось!

In [ ]:
_, ax = plt.subplots(1,1,figsize=(7,7))
Plotter.plot_fit(ax, cores[1], hp_sequence, mapping)

ax.axis("equal")
ax.axis("off")

plt.show()

# 3.1. Ради научного интереса сравним с реальными позициями $C_\alpha$!

**Замечание**: Расстояние между соседними $C_\alpha$ - 3.83 ангстрем. А между соседними узлами решетки - $\sqrt{2}$ 🤔

Поменяем масштаб для узлов решетки:

In [ ]:
pred_cas = np.array([mapping[f"L_{i}"] for i in range(len(hp_sequence))])

In [ ]:
# your code here

А еще у нас предсказанные координаты на плоскости 🤔 Добавим нулевой столбец координаты $z$:

In [ ]:
# your code here

**Готово!** Посчитаем RMSD:

In [ ]:
from Bio.PDB.QCPSuperimposer import QCPSuperimposer

In [ ]:
h_inds = [i for i in range(len(hp_sequence)) if hp_sequence[i] == 'H']

In [ ]:
imposer = QCPSuperimposer()
imposer.set(ca_coords[h_inds], pred_cas[h_inds])
imposer.run()
imposer.rms

**Вопрос для обсуждения**: хорош ли этот результат? 🤔